In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import statistics

k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

In [2]:
def my_mode( input_data ):

    freq_count = dict() 

    for d in input_data: 
        if ( d in freq_count ):
            freq_count[d] += 1 
        else: 
            freq_count[d] = 1 

    max_count = 0 
    mode_value = None 

    for k,v in freq_count.items():
        if ( v > max_count ):
            mode_value = k 
            max_count = v

    return mode_value ; 

In [3]:
def normalise_data(input_data):

    #print(input_data[0:100])
    max_value = np.argmax(input_data)
    min_value = np.argmin(input_data)
    for i in range(0, len(input_data)):
        input_data[i] = (input_data[i] - min_value)/(max_value-min_value)
    return input_data

In [8]:
def generate_data_samples( input_dataset, x_columns, y_column, sample_size):

    #y_values = [] 
    # First take the y series data and apply the compression function to get a single value per sample. 
    #y_series = input_dataset[y_column]; 
    y_values = input_dataset[y_column];
    #for i in range(0,len(input_dataset), sample_size): 

    #    if ( (i + 128) <= len(y_series)):
    #        value = y_compress_function(y_series[i:i+sample_size])
    #        y_values.append(value)
            

    # Now truncate the dataset, since we need the rows to be divisible by the sample size. 
    #input_dataset = input_dataset.truncate(after=len(y_values*sample_size)-1)

    input_dataset = input_dataset[x_columns]
    #input_dataset.replace(np.nan, 0)
    np_x_values = input_dataset.to_numpy()
    np_x_values = np_x_values.astype(np.float32)
    np_x_values = np.nan_to_num(np_x_values, 0)
    
    #Regularize the data 
    #np_x_values.shape
#    for i in range (0, np_x_values.shape[1]): 
#        normalise_data(np_x_values[:,i])     
    np_x_values.shape
    np_x_values = np_x_values.reshape(np_x_values.shape[0], np_x_values.shape[1] )
    #np_x_values = np_x_values.reshape(-1, sample_size, np_x_values.shape[1] )
    #np_x_values = np_x_values.reshape(np_x_values.shape[0], np_x_values.shape[1], np_x_values.shape[2], 1) 
    
    np_y_values = np.array(y_values); 
    np_y_values = np_y_values.reshape(np_y_values.shape[0], 1)

    return np_x_values.astype(np.float32), np_y_values.astype(np.int32) 

In [4]:
def getFileList(rootFolder, searchExp = None):

    filesFound = [] 

    for root, subdirs, files in os.walk(rootFolder): 
        for fileName in files :
            if ( (searchExp is None) or re.search(searchExp, fileName) ):
                filesFound.append(os.path.join(root,fileName)) 
    return filesFound

In [6]:
training_dataframe = pd.read_csv("./Training/3_BOS_LeftLowerLimb/rawData_Day1_l.csv")
X_train, Y_train = generate_data_samples(training_dataframe, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
np.where(np.isnan(X_train))
X_train.shape
Y_train.shape
print(len(X_train))
print(len(Y_train))

2477700
2477700


In [5]:
import os
import re
file_names = getFileList('./Training', searchExp = "rawData.*l.csv")
print(file_names)

['./Training\\10_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\10_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\13_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\13_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\15_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\15_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\16_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\16_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\17_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\17_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\3_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\3_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\4_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\4_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\5_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\5_BOS_LeftLowerLimb\\rawData_Day4_l.csv', './Training\\6_BOS_LeftLowerLimb\\rawData_Day1_l.csv', './Training\\6_BOS_LeftLowerLimb\\rawData_Day4_l.csv',

In [ ]:
for data_file in file_names:
    training_dataframe = pd.read_csv(data_file)
    print(data_file)
    X_train, Y_train = generate_data_samples(training_dataframe, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
    Y_train = Y_train.reshape(-1)
    print((X_train)[0:5])
    print((Y_train)[0:5])

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import confusion_matrix

classifier = RandomForestClassifier(n_estimators=30, max_depth=10)
for data_file in file_names:
    training_dataframe = pd.read_csv(data_file)
    print(data_file)
    X_train, Y_train = generate_data_samples(training_dataframe, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
    Y_train = Y_train.reshape(-1)
    print(len(X_train))
    print(len(Y_train))
    classifier.fit(X_train, Y_train)
    scoring = 'accuracy'
    score_RandomForest = cross_validate(classifier, X_train, Y_train, cv=k_fold, n_jobs=1, scoring=scoring)
    print(score_RandomForest)
    #model.fit(x=X_train, y=Y_train, epochs=5,class_weight=class_weight)
    

C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./Training\10_BOS_LeftLowerLimb\rawData_Day1_l.csv
2543898
2543898
{'fit_time': array([63.22971272, 62.48409796, 64.20195317, 63.16633129, 62.34449148]), 'score_time': array([1.19758201, 0.97852969, 0.99293971, 0.99049997, 0.9819293 ]), 'test_score': array([0.9961732 , 0.99617516, 0.99634223, 0.99627736, 0.99635991])}
./Training\10_BOS_LeftLowerLimb\rawData_Day4_l.csv
2174039
2174039
{'fit_time': array([33.31907368, 33.87244248, 34.28001928, 33.87290812, 34.18509245]), 'score_time': array([0.77282453, 0.76526213, 0.76395249, 0.77060795, 0.76195097]), 'test_score': array([0.99065795, 0.99073844, 0.99062345, 0.99087643, 0.99085341])}


C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./Training\13_BOS_LeftLowerLimb\rawData_Day1_l.csv
2634603
2634603
{'fit_time': array([59.61716461, 58.71348023, 59.65636301, 58.80965519, 61.50911641]), 'score_time': array([1.134094  , 1.0942781 , 1.12679744, 1.12775826, 1.11631274]), 'test_score': array([0.99007062, 0.99002887, 0.9902585 , 0.99014461, 0.99001177])}
./Training\13_BOS_LeftLowerLimb\rawData_Day4_l.csv
2451821
2451821
{'fit_time': array([47.45807362, 44.40994   , 46.25088   , 45.81290054, 46.01266289]), 'score_time': array([1.07229328, 0.99583745, 1.04406953, 0.98085237, 1.02502847]), 'test_score': array([0.99196313, 0.99184483, 0.99169393, 0.99188358, 0.99213238])}


C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


...

./Training\8_BOS_LeftLowerLimb\rawData_Day1_l.csv
2779515
2779515
{'fit_time': array([56.38405275, 57.91340709, 57.17962408, 57.81791162, 56.70398498]), 'score_time': array([1.08810997, 1.13296461, 1.10152578, 1.14007425, 1.11800313]), 'test_score': array([0.99286566, 0.99299698, 0.9931265 , 0.99272175, 0.99267138])}


C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./Training\8_BOS_LeftLowerLimb\rawData_Day4_l.csv
2160924
2160924
{'fit_time': array([43.79034185, 43.00482798, 44.9963553 , 43.72285151, 47.23154783]), 'score_time': array([0.89351726, 0.87429547, 0.85703063, 0.89098525, 0.88575459]), 'test_score': array([0.99358839, 0.99339172, 0.99346345, 0.99349815, 0.99340327])}
./Training\9_BOS_LeftLowerLimb\rawData_Day1_l.csv
2704503
2704503
{'fit_time': array([67.45607448, 68.36192799, 66.65881395, 65.99086809, 69.9596467 ]), 'score_time': array([1.30147719, 1.21541429, 1.30293584, 1.28236461, 1.23481178]), 'test_score': array([0.98228511, 0.98184696, 0.98226293, 0.98173415, 0.98242189])}


C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


./Training\9_BOS_LeftLowerLimb\rawData_Day4_l.csv
2251728
2251728
{'fit_time': array([48.8401227 , 47.06490207, 49.4425869 , 51.4289453 , 48.04434848]), 'score_time': array([0.99600744, 0.98374271, 1.09697199, 1.06909704, 1.03485346]), 'test_score': array([0.97626048, 0.97643812, 0.97657357, 0.9764958 , 0.97631372])}


In [8]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=30, max_depth=10).fit(X_train, Y_train)

scoring = 'accuracy'
score_RandomForest = cross_val_score(classifier, X_train, Y_train, cv=k_fold, n_jobs=1, scoring=scoring)
print(score_RandomForest)

C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:598: DataConvers

[0.97282359 0.97370142 0.97353392 0.97357428 0.97348145]


In [ ]:
file_names = ["./Training/3_BOS_LeftLowerLimb/rawData_Day1_l.csv", "./Training/3_BOS_LeftLowerLimb/rawData_Day4_l.csv"]

Merge Patient 10_BOS data for days 1 and 4 to create a single dataframe that will be used for validation.

In [ ]:
traindata_day1 = pd.read_csv("./Training/10_BOS_LeftLowerLimb/rawData_Day1_l.csv")
traindata_day1.head()
traindata_day4 = pd.read_csv("./Training/10_BOS_LeftLowerLimb/rawData_Day4_l.csv")
traindata_day4.head()
trainingdata = pd.concat([traindata_day1, traindata_day4], ignore_index=True)
#X_train, Y_train = generate_data_samples(trainingdata, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
print(len(trainingdata))
#trainingdata.shape
X_train = trainingdata[ ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'] ] #create a dataframe for 3 axel readings
Y_train = trainingdata['Dysk_Score'] # create a dataframe for the labels
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()
X_train = X_train.astype(np.float64)
#print(X_train[:10])
#print(Y_train[:10])
#print(len(training_labels))

In [ ]:
X_train = np.nan_to_num(X_train)
print(X_train[:10])
np.where(np.isnan(X_train))

In [ ]:
Y_train.shape

In [10]:
valdata_day1 = pd.read_csv("./12_BOS_LeftLowerLimb/rawData_Day1_l.csv")
valdata_day4 = pd.read_csv("./12_BOS_LeftLowerLimb/rawData_Day4_l.csv")
valdata_day1.head()

C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,timestamp,leftAnkle_X,leftAnkle_Y,leftAnkle_Z,leftAnkle_Magnitude,Dysk_Score
0,0,1.431424e+09,-0.15686,0,10.1961,10.1973,0
1,1,1.431424e+09,0,-0.31372,10.1961,10.2009,0
2,2,1.431424e+09,0,-0.31372,10.0392,10.0441,0
3,3,1.431424e+09,-0,-0.31373,10.0392,10.0441,0
4,4,1.431424e+09,-0.15687,-0.31372,10.1961,10.2021,0


Merge Patient 17_BOS data for days 1 and 4 to create a single dataframe that will be used for validation.

In [ ]:
valdata = pd.concat([valdata_day1, valdata_day4], ignore_index=True)
#X_val, Y_val = generate_data_samples(valdata, ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'], 'Dysk_Score', 128)
X_val = valdata[ ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'] ] #create a dataframe for 3 axel readings
Y_val = valdata['Dysk_Score'] # create a dataframe for the labels
X_val = X_val.to_numpy()
Y_val = Y_val.to_numpy()
X_val = X_val.astype(np.float64)
X_val = np.nan_to_num(X_train)
print(X_val[:10])
np.where(np.isnan(X_val))
#print(X)
#print(Y)

print(len(valdata))

In [15]:
testdata_11_day1 = pd.read_csv("./11_BOS_LeftLowerLimb/rawData_Day1_l.csv")
testdata_11_day4 = pd.read_csv("./11_BOS_LeftLowerLimb/rawData_Day4_l.csv")
testdata_18_day1 = pd.read_csv("./18_BOS_LeftLowerLimb/rawData_Day1_l.csv")
testdata_18_day4 = pd.read_csv("./18_BOS_LeftLowerLimb/rawData_Day4_l.csv")
testdata = pd.concat([testdata_11_day1, testdata_11_day4, testdata_18_day1, testdata_18_day4], ignore_index=True)
testdata.head()
X_test = testdata[ ['leftAnkle_X', 'leftAnkle_Y','leftAnkle_Z'] ] #create a dataframe for 3 axel readings
Y_test = testdata['Dysk_Score'] # create a dataframe for the labels
X_test = X_test.to_numpy()
Y_test = Y_test.to_numpy()
X_test = X_test.astype(np.float64)
X_test = np.nan_to_num(X_test)
Y_test = np.nan_to_num(Y_test)
Y_test = Y_test.reshape(-1)
print(X_test[:10])
print(Y_test[:10])

C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


[[ 0.      -0.47059 10.19608]
 [-0.15686 -0.15686 10.19608]
 [-0.      -0.31372 10.19608]
 [-0.15686 -0.15686 10.19608]
 [-0.31372 -0.15686 10.19608]
 [-0.31373 -0.15686 10.03922]
 [-0.15687 -0.15686 10.03921]
 [-0.15686 -0.31373 10.19608]
 [-0.15686 -0.15687 10.03922]
 [-0.15686 -0.15686 10.19607]]
[0 0 0 0 0 0 0 0 0 0]


In [17]:
Y_pred = classifier.predict(X_test)
print("Accuracy:", accuracy_score(Y_test, Y_pred))

Accuracy: 0.969867592434324


In [19]:
print("Precision:", accuracy_score(Y_test, Y_pred))
print("Recall:", accuracy_score(Y_test, Y_pred))
print("f1_score:", accuracy_score(Y_test, Y_pred))
print("roc_auc:", accuracy_score(Y_test, Y_pred))

Precision: 0.969867592434324
Recall: 0.969867592434324
f1_score: 0.969867592434324
roc_auc: 0.969867592434324


In [20]:
from sklearn.metrics import multilabel_confusion_matrix
mcm = multilabel_confusion_matrix(Y_test, Y_pred)
tn = mcm[:, 0, 0]
tp = mcm[:, 1, 1]
fn = mcm[:, 1, 0]
fp = mcm[:, 0, 1]
tp / (tp + fn)

array([[[      0,  298200],
        [   1837, 9657249]],

       [[9749399,    1837],
        [ 206050,       0]],

       [[9884686,       0],
        [  72600,       0]],

       [[9938986,       0],
        [  18300,       0]],

       [[9956036,       0],
        [   1250,       0]]], dtype=int64)

In [23]:
#Calculate Recall, or Sensitivity for each class
mcm = multilabel_confusion_matrix(Y_test, Y_pred)
tn = mcm[:, 0, 0]
tp = mcm[:, 1, 1]
fn = mcm[:, 1, 0]
fp = mcm[:, 0, 1]
tp / (tp + fn)
#Calculate specificity for each class:
tn / (tn + fp)

array([0.        , 0.99981161, 1.        , 1.        , 1.        ])

In [21]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', 'class 2', 'class 3', 'class 4']
print(classification_report(Y_test, Y_pred, target_names=target_names))

C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     class 0       0.97      1.00      0.98   9659086
     class 1       0.00      0.00      0.00    206050
     class 2       0.00      0.00      0.00     72600
     class 3       0.00      0.00      0.00     18300
     class 4       0.00      0.00      0.00      1250

    accuracy                           0.97   9957286
   macro avg       0.19      0.20      0.20   9957286
weighted avg       0.94      0.97      0.96   9957286



C:\Users\svetl\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
from joblib import dump, load
dump(classifier, 'RandomForest.joblib') 

['RandomForest.joblib']

In [28]:
#or it is also possible to save it via pickle.dumps - same size; the joblib is supposed to be a more efficient way
import pickle
s = pickle.dumps(classifier)
classifier2 = pickle.loads(s)
dump(classifier2, 'RandomForest2.joblib') 

['RandomForest2.joblib']

In [ ]:
#to load the model back: 
classifier = load('RandomForest.joblib')

In [4]:
pip install micromlgen

  Created wheel for micromlgen: filename=micromlgen-1.1.23-cp37-none-any.whl size=29095 sha256=8aeb7936861bf3af39ac586e39b8edaf07499007416550612b5f7e3af5a39d2d
  Stored in directory: C:\Users\svetl\AppData\Local\pip\Cache\wheels\18\8a\f0\bc837db55f7e6159d94de85c7e3b3dd78db19109f4de268f5f
Successfully built micromlgen
Note: you may need to restart the kernel to use updated packages.


In [24]:
from micromlgen import port

In [ ]:
c_code = port(classifier, classmap=classmap)
print(c_code)